## Importing libraries:
---

In [ ]:
import os
import zipfile
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from IPython.display import Image

DATA_DIR = "../raw_data/"
ZIP_PATH = os.path.join(DATA_DIR, "creditcardfraud.zip")
CSV_PATH = os.path.join(DATA_DIR, "creditcard.csv")

In [ ]:
os.makedirs(DATA_DIR, exist_ok=True)

if not os.path.exists(ZIP_PATH):
    url = "https://www.kaggle.com/api/v1/datasets/download/mlg-ulb/creditcardfraud"
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(ZIP_PATH, "wb") as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print("Succesfully downloaded.")
    else:
        print(f"Failed to download the file. Status code: {response.status_code}")

if not os.path.exists(CSV_PATH) and os.path.exists(ZIP_PATH):
    with zipfile.ZipFile(ZIP_PATH, "r") as zip_ref:
        zip_ref.extractall(DATA_DIR)
    print("Data extracted succesfully.")

if os.path.exists(CSV_PATH):
    df = pd.read_csv(CSV_PATH)
    print("DataFrame loaded succesfully.")
else:
    print("CSV file not found.")

To help understand the problems we are facing, the following image helps to determine the best metric to be used based on the problem we are solving:

![metrics.png](https://machinelearningmastery.com/wp-content/uploads/2019/12/How-to-Choose-a-Metric-for-Imbalanced-Classification-latest.png)

source: [MachineLearningMastery](https://machinelearningmastery.com/tour-of-evaluation-metrics-for-imbalanced-classification/)

In [ ]:
df.head() # Display the first few rows of the DataFrame

### Stratified splitting:
---

In [ ]:
# Split the DataFrame into features and target variable
# 'Class' is the target variable indicating fraud (1) or not fraud (0)
# The rest of the columns are features used for prediction
X = df.drop('Class', axis=1)
y = df['Class']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size = 0.3, random_state = 42)

### Oversample/undersample before or after splitting data?
---
Main rule: **Always** after.

### Feature Scaling:
---

In [ ]:
from typing import List
from sklearn.preprocessing import StandardScaler

# Creating function for scaling
def Standard_Scaler(df: pd.DataFrame, col_names: List[str]) -> pd.DataFrame:
    """
    Standardizes the features in the DataFrame using StandardScaler.
    Args:
        df (pd.DataFrame): The DataFrame containing the features to be scaled.
        col_names (list): List of column names to be scaled.
    Returns:
        pd.DataFrame: The DataFrame with the specified columns scaled.
    """
    features = df[col_names]
    scaler = StandardScaler().fit(features.values)
    features = scaler.transform(features.values)
    df[col_names] = features
    
    return df

In [ ]:
X_train = Standard_Scaler (X_train, ['Amount'])
X_test = Standard_Scaler (X_test, ['Amount'])

## Exploratory Data Analysis:
---

In [ ]:
labels=["Genuine","Fraud"]

fraud_or_not = df["Class"].value_counts().tolist()
values = [fraud_or_not[0], fraud_or_not[1]]

fig = px.pie(values=df['Class'].value_counts(), names=labels , width=700, height=400, color_discrete_sequence=["skyblue","black"]
             ,title="Fraud vs Genuine transactions")
fig.show()

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier

#We are going to ensure that we have the same splits of the data every time. 
#We can ensure this by creating a KFold object, kf, and passing cv=kf instead of the more common cv=5.

kf = StratifiedKFold(n_splits=5, shuffle=False)

In [ ]:
rf = RandomForestClassifier(n_estimators=100, random_state=13)
#cross_val_score(rf, X_train, y_train, cv=kf, scoring='recall')

In [ ]:
from sklearn.model_selection import cross_val_score

score = cross_val_score(rf, X_train, y_train, cv=kf, scoring='recall')
print("Cross Validation Recall scores are: {}".format(score))
print("Average Cross Validation Recall score: {}".format(score.mean()))